# Read the PDF

In [18]:
# libraries for text cleaning
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as sw
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer

# library to read the PDF
%pip install feedparser
import pdfplumber

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/henrytran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/henrytran/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [19]:
# library to read the PDF
%pip install feedparser
import pdfplumber

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [25]:
# We should use this code
# this is the text extracted from the first 5 pages extracted from the
def raw_data_extraction(input_pdf):
    # take the pdf as an input and return the whole pdf as text
    raw_input=""
    with pdfplumber.open(input_pdf) as pdf:
        for page in pdf.pages:
            raw_input+=(page.extract_text() or "") + "\n"
    return raw_input

def article_extraction(raw_input):
    #all_news = re.sub(r'\([^)]*\)', '', raw_input)
    #sww=set(sw.words())
    article=""
    details=""
    article_status=False # mark the start of the article
    details_status=False
    for row in raw_input.lower().split('\n'):
        if row=='full text':
            article_status=True
            continue
        if article_status:
            if row=='details':
                article_status=False
                details_status=True
                continue
            article+=row+" "
        if details_status:
            if row=='links':
                details_status=False
                continue
            details+=row+'\n'
    return article, details

# temporaryly do not need this
def clean_article(article):
    lemmatizer=WordNetLemmatizer()
    desc = re.sub(r'\([^)]*\)', '', article)
    sent_desc =sent_tokenize(desc)
    sww=set(sw.words())
    abstraction = []
    for sent in sent_desc:
        tokens = re.sub(r"[^a-z0-9]+", " ", sent.lower())
        words = word_tokenize(tokens)
        remaining_words = [word for word in words if word not in sww]
        if remaining_words:
            lemmatised_words=[]
            for word in remaining_words:
                lemmatised_word=lemmatizer.lemmatize(word)
                lemmatised_words.append(lemmatised_word)
            abstraction.append(lemmatised_words)
    final_doc=""
    for sent in abstraction[:-1]:
        for word in sent:
            final_doc+=word+" "
    return final_doc # remove the name of the writer

raw_input1=raw_data_extraction('3pages.pdf')
article1, details1=article_extraction(raw_input1)
doc1=clean_article(article1)
print(raw_input1)

Document 1 of 222
U.S. News: BLS Pick Has Forged Partisan Path ---
Nominee has praised Trump's policies but lacks the
experience of his predecessors
Kiernan, Paul . Kiernan, Paul.
ProQuest document link
FULL TEXT
Conservative economist Erwin John "E.J." Antoni sometimes jokes on social media that the "L" in BLS is silent.
President Trump this week tapped Antoni to run the Bureau of Labor Statistics, the agency whose data and
methodologies he has long criticized, especially when it produces numbers that Trump doesn't like. Antoni recently
proposed suspending the monthly jobs report, one of the most important data releases for the economy and
markets. On Tuesday, a White House official noted that Antoni made the comment before he knew he was going to be
chosen and that his comments don't reflect official BLS policy.
If confirmed by the Senate, Antoni would run a 141-year-old agency staffed by around 2,000 economists, statisticians
and other officials. The BLS has a long record of indepen

In [29]:
details1.split('\n')

['subject: conservatism; fiscal policy; presidents; economists; public policy; economics;',
 'congressional committees',
 'business indexing term: subject: fiscal policy economists economics; corporation: linkedin corp; industry:',
 '92111 : executive offices 92112 : legislative bodies',
 'location: texas; united states--us',
 'people: trump, donald j; antoni, erwin john iii',
 'company / organization: name: linkedin corp; naics: 518210; name: bureau of labor statistics; naics: 921110,',
 '923110; name: texas public policy foundation; naics: 813211; name: heritage',
 'foundation-washington dc; naics: 541720',
 'classification: 92111: executive offices; 92112: legislative bodies',
 'publication title: wall street journal, eastern edition; new york, n.y.',
 'first page: a2',
 'publication year: 2025',
 'publication date: aug 14, 2025',
 'publisher: dow jones &company inc.',
 'place of publication: new york, n.y.',
 'country of publication: united states',
 'publication subject: business 

# Create the set of fiscal policy related words

In [13]:
# the set of recurring words in these articles related to fiscal policy
fiscal_terms=['tariffs', 'tax cuts', 'budget deficits', 'presidents', 'fiscal policy', 'political campaigns', 'federal budget', 'economic growth', 'interest rates',
                'tax increases', 'gross domestic product-gdp', 'tax rates', 'economists', 'government spending', 'investments', 'voters', 'inflation', 'legislators', 'taxes',
                'federal reserve monetory policy', 'investors', 'presidential elections', 'congressional committees', 'borrowing', 'central banks', 'immigration policy', 
                'international trade', 'economic conditions', 'legislation', 'american dollar', 'social security', 'dow jones averages', 'pandemics', 'treasuries', 'costs',
                'funding', 'manufacturing', 'political leadership', 'clean technology', 'national debt', 'national security', 'stocks', 'exports', 'immigrants',
                'leadership', 'medicare', 'political parties', 'prime ministers', 'scandals', 'tax credits', 'trade policy', 'border security', 'consumer price index',
                'defense spending', 'economic impact', 'economic policy', 'employees', 'federal legislation', 'households', 'immigration', 'international relations', 'low income groups',
                'prices', 'social networks', 'tax legislation', 'trade disputes', 'budgets', 'consumers', 'corporate profits', 'currency', 'deportation', 
                'infrastructure', 'international economic relations', 'nominations', 'nvidia corp', 'polls & surveys', 'provisions', 'recessions', 'securities markets',
                'trade relations', 'bond markets', 'cities', 'consumptions', 'cost control', 'decision making', 'deficit financing', 'economic crisis', 'economic development enforcement', 
                'factories', 'federal funding', 'global economy', 'government bonds', 'income taxes', 'international relations-us', 'profits', 'stock exchanges', 'supply chains', 'tax refunds']

In [14]:
# group all terms into groups
terms_groups={'tax': ['tax cuts','tax increases', 'tax rates', 'tax credits', 'taxes', 'tax legislation', 'income taxes', 'tax refunds', 'corporate profits', 'profits'],
                'tariffs_trade': ['tariffs', 'trade policy', 'trade disputes','international trade','trade relations', 'exports','supply chains', 'border security', 'international relations-us', 'international relations', 'immigration policy', 'immigrants', 'immigration', 'deportation'],
                'budget_debt': ['budget deficits', 'federal budget', 'budgets', 'deficit financing', 'national debt', 'treasuries', 'borrowing', 'federal funding', 'funding', 'cost control', 'costs', 'investments', 'provisions'],
                'spending_social_program': ['government spending', 'defense spending', 'infrastructure', 'social security', 'medicare', 'federal legislation', 'households', 'low income groups', 'national security', 'pandemics', 'manufacturing'],
                'monetary_financial_policy': ['government bonds', 'bond markets', 'federal reserve monetory policy', 'central banks', 'stocks', 'stock exchanges', 'dow jones averages','american dollar', 'currency', 'securities markets', 'fiscal policy', 'interest rates', 'inflation', 'consumer price index', 'prices', 'recessions', 'enforcement'],
                'economy': ['economists', 'economic growth', 'economic conditions', 'economic policy', 'economic impact', 'international economic relations', 'global economy', 'economic crisis', 'economic development', 'gross domestic product-gdp', 'economic development enforcement'],
                'other_context': ['presidents', 'investors', 'political campaigns', 'political leadership', 'political parties', 'legislation','congressional committees','prime ministers','scandals','nominations', 'clean technology', 'cities', 'voters', 'legislators', 'leadership', 'consumers', 'social networks', 'presidential elections', 'nvidia corp', 'polls & surveys', 'consumptions', 'decision making','employees', 'factories']}

In [15]:
# check for missing words not in the dictionary -  ensure it could not print anything
check_list=[]
for list in terms_groups.values():
    for word in list:
        check_list.append(word)

for word in fiscal_terms:
    if word not in check_list:
        print(word)


# Match the terms with 

In [21]:
# match the fiscal terms to the document
def count_matching_words(terms_list, document):
    terms_count={}
    for term in terms_list:
        term_count=document.count(term)
        if term_count >0:
            terms_count[term]=term_count
    return terms_count

count_matching_words(fiscal_terms,doc1)

{'fiscal policy': 2, 'inflation': 1, 'consumer price index': 1}